In [11]:
from src.data_reader import DataReader
from src.constant import *
import matplotlib.pyplot as plt
import numpy as np
from src.datasets.dataset import Dataset
from src.models.rp3b.rp3b import Rp3b
import pandas as pd
from src.evaluation import compute_mrr

In [12]:
dataset = Dataset()

In [13]:
split_dict = dataset.get_split()
train, train_label = split_dict[TRAIN]
val, val_label = split_dict[VAL]
test, test_label = split_dict[TEST]

val_test = pd.concat([val, test])
val_test_label = pd.concat([val_label, test_label])

In [28]:
from datetime import datetime, timedelta
max_date = train[DATE].max()
train_limit_date = max_date - timedelta(days=120)
filtered_train = train[train[DATE] > train_limit_date].copy()
id_filtered_train = filtered_train[SESS_ID].unique()

final_train_data = train[train[SESS_ID].isin(id_filtered_train)]
final_train_label = train_label[train_label[SESS_ID].isin(id_filtered_train)]

In [29]:
full_data = dataset.get_train_sessions()
# we can not use that for the final submission
lead_data = dataset.get_test_leaderboard_sessions()
final_data = dataset.get_test_final_sessions()

In [30]:
train_pur = pd.concat([final_train_data, final_train_label], axis=0)

In [31]:
model = Rp3b(dataset, topk=1000, alpha=0.7, beta=0.3, time_weight=50)

In [32]:
model.compute_similarity_matrix(train_pur)

Done: 100%|██████████| 23691/23691 [00:00<00:00, 34179.44it/s]


In [33]:
recs = model.recommend(
    interactions=val_test,
    remove_seen=True,
    cutoff=100,
    leaderboard=False
)

Using Time Weight on Interaction matrix


In [34]:
compute_mrr(recs, val_test_label)

MRR: 0.16773174645630703


0.16773174645630703

In [ ]:
recs.reset_index(drop=True).to_feather(dataset.get_train_recs_df_folder() / "rp3b.feather")

In [ ]:
concat_full_data = pd.concat([final_train_data, final_train_label, val, val_label, test, test_label], axis=0)

In [ ]:
model = Rp3b(dataset, topk=1000, alpha=0.7, beta=0.3, time_weight=None)

In [ ]:
model.compute_similarity_matrix(concat_full_data)

In [ ]:
recs_lead = model.recommend(
    interactions=lead_data,
    remove_seen=True,
    cutoff=100,
    leaderboard=True
)

In [ ]:
recs_lead.reset_index(drop=True).to_feather(dataset.get_leaderboard_recs_df_folder() / "rp3b.feather")

In [ ]:
recs_final = model.recommend(
    interactions=final_data,
    remove_seen=True,
    cutoff=100,
    leaderboard=True
)

In [ ]:
recs_final.reset_index(drop=True).to_feather(dataset.get_final_recs_df_folder() / "rp3b.feather")